# KNMI Data Pull: Download and Save Precipitation & Evaporation Series

This notebook downloads daily weather data from KNMI for a given station and date range,
and saves the precipitation and evapotranspiration series as separate CSV files.

- Output:
    - Precipitation → `input_files/input_prec/prec_station_<station>.csv`
    - Evapotranspiration → `input_files/input_evap/evap_station_<station>.csv`

You can adjust the station and date range below.

In [1]:
# --- User settings ---
station = 249  # KNMI station number (e.g., 249 for Berkhout)
start_date = '2013-01-01'  # Start date (YYYY-MM-DD)
end_date = '2025-7-10'    # End date (YYYY-MM-DD)

In [2]:
# --- Imports and utility function ---
import os
import numpy as np
import pandas as pd
import requests
from io import StringIO

def fetch_knmi_prec_evap(station: int, start_date: str, end_date: str):
    """
    Fetch KNMI daily data and compute precipitation and evapotranspiration (Hargreaves).
    
    Parameters:
    - station (int): KNMI station number (e.g., 249 for Berkhout)
    - start_date (str): Start date in 'YYYY-MM-DD' or 'YYYYMMDD' format
    - end_date (str): End date in 'YYYY-MM-DD' or 'YYYYMMDD' format
    
    Returns:
    - prec (pd.Series): Precipitation series in mm/day (float64)
    - evap (pd.Series): Evapotranspiration series in mm/day (float64)
    """
    # Convert dates to YYYYMMDD if needed
    start = start_date.replace('-', '')
    end = end_date.replace('-', '')
    url = 'https://www.daggegevens.knmi.nl/klimatologie/daggegevens'
    params = {
        'start': start,
        'end': end,
        'stns': str(station),
        'vars': 'Q:RH:TN:TX:TG',
        'fmt': 'csv'
    }
    response = requests.post(url, data=params)
    response.raise_for_status()
    csv_data = '\n'.join(line for line in response.text.splitlines() if not line.startswith('#'))


    knmi_df = pd.read_csv(StringIO(csv_data), header=None)
    knmi_df.columns = ['STN', 'DATE', 'Q', 'RH', 'TN', 'TX', 'TG']
    knmi_df['DATE'] = pd.to_datetime(knmi_df['DATE'], format='%Y%m%d')
    knmi_df['Q'] = knmi_df['Q'] * 0.01
    knmi_df['RH'] = knmi_df['RH'] / 10.0
    knmi_df['TN'] = knmi_df['TN'] / 10.0
    knmi_df['TX'] = knmi_df['TX'] / 10.0
    knmi_df['TG'] = knmi_df['TG'] / 10.0
    knmi_df.rename(columns={
        'Q': 'Radiation',
        'RH': 'Precipitation',
        'TN': 'Tmin',
        'TX': 'Tmax',
        'TG': 'Tavg'
    }, inplace=True)
    def hargreaves_pet(row):
        t_avg, t_max, t_min, ra = row['Tavg'], row['Tmax'], row['Tmin'], row['Radiation']
        if np.isnan(t_avg) or np.isnan(t_max) or np.isnan(t_min) or np.isnan(ra):
            return np.nan
        return 0.0023 * (t_avg + 17.8) * np.sqrt(t_max - t_min) * ra
    knmi_df['ET'] = knmi_df.apply(hargreaves_pet, axis=1)
    knmi_df = knmi_df.set_index('DATE')
    prec = knmi_df['Precipitation'].astype(float)
    evap = knmi_df['ET'].astype(float)
    return prec, evap

In [3]:
# --- Ensure output directories exist ---
import os
prec_dir = '../input_files/input_prec'
evap_dir = '../input_files/input_evap'
os.makedirs(prec_dir, exist_ok=True)
os.makedirs(evap_dir, exist_ok=True)

In [4]:
# --- Download and save data ---
prec, evap = fetch_knmi_prec_evap(station, start_date, end_date)

# Save precipitation
prec_path = f'{prec_dir}/prec_station_{station}.csv'
prec.to_csv(prec_path, header=True)
print(f'Precipitation saved to: {prec_path}')

# Save evapotranspiration
evap_path = f'{evap_dir}/evap_station_{station}.csv'
evap.to_csv(evap_path, header=True)
print(f'Evapotranspiration saved to: {evap_path}')

Precipitation saved to: ../input_files/input_prec/prec_station_249.csv
Evapotranspiration saved to: ../input_files/input_evap/evap_station_249.csv


---

*You can now use these CSV files as input for further analysis or modeling workflows.*